In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import re
from nltk.tokenize import sent_tokenize,word_tokenize,regexp_tokenize,TweetTokenizer
import matplotlib.pyplot as plt
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.models.tfidfmodel import TfidfModel
from gensim.corpora.dictionary import Dictionary
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import spacy
from bs4 import BeautifulSoup

C:\Users\saheediyanda\Anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
data = pd.read_csv('BA_Review')
data.head()

,Unnamed: 0,reviews
0,0,✅ Trip Verified | I have come to boarding and...
1,1,✅ Trip Verified | Stinking nappies being chang...
2,2,✅ Trip Verified | Worst service ever. Lost bag...
3,3,✅ Trip Verified | BA 246 21JAN 2023 Did not a...
4,4,✅ Trip Verified | Not a great experience. I co...


In [3]:
data.shape

(3742, 2)

In [4]:
data['verified?'] = data['reviews'].apply(lambda x: x.split('|')[0])

In [5]:
data

,Unnamed: 0,reviews,verified?
0,0,✅ Trip Verified | I have come to boarding and...,✅ Trip Verified
1,1,✅ Trip Verified | Stinking nappies being chang...,✅ Trip Verified
2,2,✅ Trip Verified | Worst service ever. Lost bag...,✅ Trip Verified
3,3,✅ Trip Verified | BA 246 21JAN 2023 Did not a...,✅ Trip Verified
4,4,✅ Trip Verified | Not a great experience. I co...,✅ Trip Verified
...,...,...,...
3737,3737,Flew LHR - VIE return operated by bmi but BA a...,Flew LHR - VIE return operated by bmi but BA a...
3738,3738,LHR to HAM. Purser addresses all club passenge...,LHR to HAM. Purser addresses all club passenge...
3739,3739,My son who had worked for British Airways urge...,My son who had worked for British Airways urge...
3740,3740,London City-New York JFK via Shannon on A318 b...,London City-New York JFK via Shannon on A318 b...


In [6]:
data.loc[data['verified?'] != '✅ Trip Verified ','verified?'] = 'Not Verified'
data.loc[data['verified?'] == '✅ Trip Verified ','verified?'] = 'Trip Verified'

In [7]:
data

,Unnamed: 0,reviews,verified?
0,0,✅ Trip Verified | I have come to boarding and...,Trip Verified
1,1,✅ Trip Verified | Stinking nappies being chang...,Trip Verified
2,2,✅ Trip Verified | Worst service ever. Lost bag...,Trip Verified
3,3,✅ Trip Verified | BA 246 21JAN 2023 Did not a...,Trip Verified
4,4,✅ Trip Verified | Not a great experience. I co...,Trip Verified
...,...,...,...
3737,3737,Flew LHR - VIE return operated by bmi but BA a...,Not Verified
3738,3738,LHR to HAM. Purser addresses all club passenge...,Not Verified
3739,3739,My son who had worked for British Airways urge...,Not Verified
3740,3740,London City-New York JFK via Shannon on A318 b...,Not Verified


In [8]:
data['reviews'][0]

'✅ Trip Verified |  I have come to boarding and my cabin luggage was taken, "because the plane is full". I asked to take it to the cabin, because I have large notebook and  electronics in it, but nothing changes. So, now I am sitting in the plane, holding notebook and several packs from my luggage, and think of how I will stand with all this stuff in my hands in a passport control line in Warsaw. Other people around me have come to the plane with cabin luggage!'

In [9]:
data['reviews'][3]

"✅ Trip Verified |\xa0\xa0BA 246 21JAN 2023 Did not appreciate the unprofessional attitude of the pilots. Flight scheduled departure 16:20. Advised boarding time 15:20. Whole flight full of passengers waiting at the gate to board at 15:20. 15:40 the cabin crew board. 15:55 the pilots board - each with a Sao Paulo Airport Duty Free branded shopping bag. 16:20 the flight is still boarding. Finally pushes back just before 17:00, 40 minutes late. Captain came on the intercom to announce the delay was due to 'traffic between the crew hotel and the airport'. Sorry Captain the whole plane saw you and your pilot colleagues board fifteen minutes after the cabin crew clutching your duty free that you and your pilot colleagues still made time to stop for."

In [10]:
# check the value count of the verified column
data['verified?'].value_counts()

Not Verified     2558
Trip Verified    1184
Name: verified?, dtype: int64

In [11]:
import re, nltk, spacy, gensim
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from pprint import pprint

# Plotting tools
import matplotlib.pyplot as plt

This code defines a function called `sent_to_words` that takes a list of sentences as input. The function iterates over each sentence in the input list and uses the `gensim.utils.simple_preprocess` function to process the sentence by converting it to lowercase and removing any punctuation. The processed sentence is then yielded back.

The `data_words` variable is then assigned the result of calling the `sent_to_words` function with the 'review_text' column of the `data` dataframe as input. This will convert the sentences in the 'review_text' column into a list of words.

Finally, the code prints the first element of the `data_words` list. This will display the processed words from the first sentence in the 'review_text' column of the `data` dataframe.

In [13]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data['reviews'].tolist()))

print(data_words[:1])

[['trip', 'verified', 'have', 'come', 'to', 'boarding', 'and', 'my', 'cabin', 'luggage', 'was', 'taken', 'because', 'the', 'plane', 'is', 'full', 'asked', 'to', 'take', 'it', 'to', 'the', 'cabin', 'because', 'have', 'large', 'notebook', 'and', 'electronics', 'in', 'it', 'but', 'nothing', 'changes', 'so', 'now', 'am', 'sitting', 'in', 'the', 'plane', 'holding', 'notebook', 'and', 'several', 'packs', 'from', 'my', 'luggage', 'and', 'think', 'of', 'how', 'will', 'stand', 'with', 'all', 'this', 'stuff', 'in', 'my', 'hands', 'in', 'passport', 'control', 'line', 'in', 'warsaw', 'other', 'people', 'around', 'me', 'have', 'come', 'to', 'the', 'plane', 'with', 'cabin', 'luggage']]


This code defines a function called `lemmatization` that takes a list of texts and an optional parameter `allowed_postags` as input. The `allowed_postags` parameter specifies the parts of speech that should be included in the lemmatization process and defaults to nouns, adjectives, verbs, and adverbs if not specified.

Inside the function, a new list called `texts_out` is initialized. Then, for each text in the input list, the function uses the spaCy `nlp` object to process the text and tokenize it into words. The words are then lemmatized and appended to the `texts_out` list, based on the allowed parts of speech specified.

Finally, the function returns the `texts_out` list containing the lemmatized texts.

The lemmatization process involves converting words to their base or dictionary form, which can help in standardizing and normalizing the text data for further analysis or natural language processing tasks.

In [14]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

['trip verify come boarding cabin luggage take plane full ask take cabin large notebook electronic change so now sit plane hold notebook several pack luggage think stand stuff hand passport control line other people come plane cabin luggage']


In [16]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)

The code performs the following actions:

1. It converts a sparse matrix `data_vectorized` into a dense matrix `data_dense` using the `todense()` method. In the context of natural language processing, this is often done after vectorizing text data using techniques like TF-IDF or Count Vectorization, which typically result in sparse matrices.

2. After converting the matrix, the code calculates the sparsity of the matrix. Sparsity is a measure of how many elements in a matrix are zero. In this case, the code calculates the sparsity as the percentage of non-zero elements in the matrix. It does this by comparing each element in the dense matrix to zero, summing the number of non-zero elements, and then dividing by the total number of elements in the matrix.

3. The result is printed, showing the sparsity as a percentage.

In summary, the code first converts a sparse matrix to a dense matrix, and then calculates and prints the sparsity of the dense matrix. This kind of analysis is common in text processing and machine learning to understand the density of the data and to make decisions about the most appropriate algorithms and techniques to use.

In [17]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  2.320316755299133 %


In [19]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=4,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_output) 

[[0.00893255 0.26482893 0.71730119 0.00893733]
 [0.25440386 0.66780823 0.00142746 0.07636045]
 [0.82670977 0.01124806 0.15097765 0.01106452]
 ...
 [0.53624558 0.34885259 0.00417578 0.11072605]
 [0.01724364 0.01871156 0.01712368 0.94692112]
 [0.00512972 0.59978365 0.00520229 0.38988434]]


This code is using the Latent Dirichlet Allocation (LDA) algorithm, which is a popular technique for topic modeling in natural language processing. Here's a breakdown of the code:

1. `LatentDirichletAllocation` is a class from the `sklearn.decomposition` module that represents an LDA model. It takes several parameters:

   - `n_components`: Specifies the number of topics to be identified in the data.
   - `max_iter`: Sets the maximum number of iterations for the LDA algorithm to converge.
   - `learning_method`: Determines the method used for learning the model. In this case, 'online' learning method is used.
   - `random_state`: Sets the seed for the random number generator, ensuring reproducibility of the results.
   - `batch_size`: Defines the number of documents to be used in each learning iteration.
   - `evaluate_every`: Specifies how often the model should compute the perplexity. A value of -1 means the perplexity is not computed.
   - `n_jobs`: Sets the number of CPU cores to use for parallelizing the computation. A value of -1 means all available CPU cores will be used.

2. The `LatentDirichletAllocation` model is then fitted to the `data_vectorized` input, which is typically a document-term matrix obtained through techniques like TF-IDF or Count Vectorization. The `fit_transform` method applies the LDA model to the input data and returns the document-topic distribution.

After running this code, `lda_output` will contain the document-topic matrix, where each row represents a document and each column represents a topic, with the values indicating the probability of the document belonging to that topic. This can be further analyzed to understand the topics present in the corpus of documents.

In [20]:
# Create Document - Topic Matrix
lda_output = lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,dominant_topic
Doc0,0.010000,0.260000,0.720000,0.010000,2
Doc1,0.250000,0.670000,0.000000,0.080000,1
Doc2,0.830000,0.010000,0.150000,0.010000,0
Doc3,0.000000,0.000000,0.990000,0.000000,2
Doc4,0.010000,0.010000,0.760000,0.230000,2
Doc5,0.030000,0.480000,0.140000,0.350000,1
Doc6,0.860000,0.130000,0.000000,0.000000,0
Doc7,0.100000,0.880000,0.010000,0.010000,1
Doc8,0.490000,0.150000,0.230000,0.130000,0
Doc9,0.370000,0.080000,0.540000,0.010000,2


In [21]:
df_document_topic.shape

(3742, 5)

In [22]:
topics_ = df_document_topic['dominant_topic'].tolist()
data['topic'] = topics_
data.head()

,Unnamed: 0,reviews,verified?,topic
0,0,✅ Trip Verified | I have come to boarding and...,Trip Verified,2
1,1,✅ Trip Verified | Stinking nappies being chang...,Trip Verified,1
2,2,✅ Trip Verified | Worst service ever. Lost bag...,Trip Verified,0
3,3,✅ Trip Verified | BA 246 21JAN 2023 Did not a...,Trip Verified,2
4,4,✅ Trip Verified | Not a great experience. I co...,Trip Verified,2


The code you've provided is related to sentiment analysis using the VADER (Valence Aware Dictionary and sEntiment Reasoner) tool, which is a lexicon and rule-based sentiment analysis tool specifically designed for analyzing sentiments in text. Here's a breakdown of the code:

1. `SentimentIntensityAnalyzer` is a class from the `nltk.sentiment.vader` module. It represents the VADER sentiment analysis tool. In the first line of the code, you're instantiating a new `SentimentIntensityAnalyzer` object and assigning it to the variable `sid`.

2. Once the `SentimentIntensityAnalyzer` object is created, the `apply` method is used on the 'reviews' column of the `data` DataFrame. This method applies the `polarity_scores` function from the `sid` object to each review in the 'reviews' column. The `polarity_scores` function returns a dictionary containing the sentiment scores for each review, including the positive, negative, neutral, and compound scores.

After running this code, the `sentiment_scores` variable will contain a series of dictionaries, with each dictionary representing the sentiment scores for a specific review in the 'reviews' column of the `data` DataFrame. These scores can then be used to analyze the overall sentiment of the reviews or to perform further sentiment analysis tasks.

In [23]:
# Load SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Instantiate new SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

# Generate sentiment scores
sentiment_scores = data['reviews'].apply(sid.polarity_scores)

In [36]:
sentiment_scores

{'positive': 1, 'negative': -1}

In [38]:
sentiment_scores = {"positive": 1, "negative": -1}

# Assuming your data is in a DataFrame named 'data' with a column named 'sentiment'
data['sentiment_score'] = data['reviews'].apply(lambda x: sentiment_scores.get(x))

# Now your DataFrame has a new column 'sentiment_score' with numerical values indicating sentiment


In [ ]:
from textblob import TextBlob

# Assuming your data is in a DataFrame named 'data' with a column named 'sentiment'
data['sentiment_score'] = data['sentiment'].apply(lambda x: TextBlob(x).sentiment.polarity)

# Now your DataFrame has a new column 'sentiment_score' with numerical values indicating sentiment

In [29]:
text = "I had a fantastic movie experience! The acting was superb, and the plot kept me on the edge of my seat."

cleaned_text = text.lower().replace(",", "").replace(".", "").split()
print(cleaned_text)

['i', 'had', 'a', 'fantastic', 'movie', 'experience!', 'the', 'acting', 'was', 'superb', 'and', 'the', 'plot', 'kept', 'me', 'on', 'the', 'edge', 'of', 'my', 'seat']


In [31]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
features = vectorizer.fit_transform(cleaned_text)
print(features.toarray())

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]]


In [34]:
vectorizer = CountVectorizer()
features = vectorizer.fit_transform(cleaned_text)

In [35]:
features

<21x17 sparse matrix of type '<class 'numpy.int64'>'
	with 19 stored elements in Compressed Sparse Row format>

In [25]:
from textblob import TextBlob



text = ''' The movie was great. The movie was bad. The movie was really bad 
'''
textblob = TextBlob()
blob = textblob(text)

for sentence in blob.sentences:
    print(sentence.sentiment.polarity)

TypeError: __init__() missing 1 required positional argument: 'text'

In [41]:
!pip install textblob

     -------------------------------------- 636.8/636.8 kB 1.2 MB/s eta 0:00:00


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
